In [42]:
import json
import pandas as pd
from datetime import datetime

In [43]:
file_path = "../../data/raw/NepalStock_top_all_20250227.json"

with open(file_path, "r", encoding="utf-8") as file:
    json_data = json.load(file)
print(json_data)


[{'title': 'Nepali 🇳🇵 Share market', 'score': 194, 'id': 'lpgvdk', 'url': 'https://i.redd.it/pij7swzm1zi61.jpg', 'author': 'Stockphotographer', 'body': '', 'created_utc': '2021-02-22T12:01:50', 'num_comments': 9, 'permalink': '/r/NepalStock/comments/lpgvdk/nepali_share_market/', 'flair': 'Memes/Fun/Laughs', 'nsfw': False, 'awards': 0}, {'title': 'This is how hydro bull is going on', 'score': 170, 'id': 'ozkzj1', 'url': 'https://v.redd.it/9mw10nr4luf71', 'author': 'BeingBuddha21', 'body': '', 'created_utc': '2021-08-07T08:19:42', 'num_comments': 20, 'permalink': '/r/NepalStock/comments/ozkzj1/this_is_how_hydro_bull_is_going_on/', 'flair': 'Memes/Fun/Laughs', 'nsfw': False, 'awards': 0}, {'title': 'The GameStop stock and r/wallstreetbets case explained.', 'score': 157, 'id': 'l7hrqb', 'url': 'https://i.redd.it/6dsqv12xw6e61.jpg', 'author': 'None', 'body': '', 'created_utc': '2021-01-29T09:08:58', 'num_comments': 25, 'permalink': '/r/NepalStock/comments/l7hrqb/the_gamestop_stock_and_rwall

In [44]:
df = pd.DataFrame(json_data)

In [45]:
df.head()

,title,score,id,url,author,body,created_utc,num_comments,permalink,flair,nsfw,awards
0,Nepali 🇳🇵 Share market,194,lpgvdk,https://i.redd.it/pij7swzm1zi61.jpg,Stockphotographer,,2021-02-22T12:01:50,9,/r/NepalStock/comments/lpgvdk/nepali_share_mar...,Memes/Fun/Laughs,False,0
1,This is how hydro bull is going on,170,ozkzj1,https://v.redd.it/9mw10nr4luf71,BeingBuddha21,,2021-08-07T08:19:42,20,/r/NepalStock/comments/ozkzj1/this_is_how_hydr...,Memes/Fun/Laughs,False,0
2,The GameStop stock and r/wallstreetbets case e...,157,l7hrqb,https://i.redd.it/6dsqv12xw6e61.jpg,None,,2021-01-29T09:08:58,25,/r/NepalStock/comments/l7hrqb/the_gamestop_sto...,Misc,False,0
3,NEPSE TaskBar widget for Windows that I've bee...,151,ngusl2,https://i.redd.it/u2l61cz2l8071.png,chaosifier,,2021-05-20T14:23:10,47,/r/NepalStock/comments/ngusl2/nepse_taskbar_wi...,Softwares,False,0
4,Revenant amoung most new investors these days.,151,nfz0pn,https://i.redd.it/jrqss65231071.jpg,randomcommenter7,,2021-05-19T13:03:55,7,/r/NepalStock/comments/nfz0pn/revenant_amoung_...,Memes/Fun/Laughs,False,0


In [46]:
# Feature engineering
df['created_utc'] = pd.to_datetime(df['created_utc'])
df['created_hour'] = df['created_utc'].dt.hour
df['created_dayofweek'] = df['created_utc'].dt.dayofweek

In [47]:
# Combine text fields (title + body) for NLP models
df['combined_text'] = df['title'].fillna('') + ' ' + df['body'].fillna('')

In [48]:
# Clean URLs and special chars from text
df['cleaned_text'] = df['combined_text'].str.replace(r'http\S+', '', regex=True).str.replace(r'\W+', ' ', regex=True)

In [49]:
# Encode categorical data
df['flair_encoded'] = pd.factorize(df['flair'])[0]


In [50]:
# Drop irrelevant columns
df = df.drop(columns=['url', 'id', 'permalink', 'created_utc', 'author'])

In [51]:
df.head()

,title,score,body,num_comments,flair,nsfw,awards,created_hour,created_dayofweek,combined_text,cleaned_text,flair_encoded
0,Nepali 🇳🇵 Share market,194,,9,Memes/Fun/Laughs,False,0,12,0,Nepali 🇳🇵 Share market,Nepali Share market,0
1,This is how hydro bull is going on,170,,20,Memes/Fun/Laughs,False,0,8,5,This is how hydro bull is going on,This is how hydro bull is going on,0
2,The GameStop stock and r/wallstreetbets case e...,157,,25,Misc,False,0,9,4,The GameStop stock and r/wallstreetbets case e...,The GameStop stock and r wallstreetbets case e...,1
3,NEPSE TaskBar widget for Windows that I've bee...,151,,47,Softwares,False,0,14,3,NEPSE TaskBar widget for Windows that I've bee...,NEPSE TaskBar widget for Windows that I ve bee...,2
4,Revenant amoung most new investors these days.,151,,7,Memes/Fun/Laughs,False,0,13,2,Revenant amoung most new investors these days.,Revenant amoung most new investors these days,0


In [56]:
import os

# Ensure the directory exists
save_dir = r"../../data/processed"

# Save to CSV
csv_path = os.path.join(save_dir, 'preprocessed_data.csv')
df.to_csv(csv_path, index=False)
